<a href="https://colab.research.google.com/github/prith27/Best_regression_algol/blob/main/6_Module_2_Part_II_Query_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture Module 2.2: Making Choices about Data Processing

## LinkedIn Social Analysis

Our next module explores concepts in:

* Algorithmic implications of design choices
* Techniques for indexing, parallelism, and sequence

It sets the stage for Module 3, which focuses on cloud/cluster-compute data processing.



In [1]:
!wget -nc https://storage.googleapis.com/penn-cis5450/linkedin_anon.jsonl

--2025-09-29 18:55:41--  https://storage.googleapis.com/penn-cis5450/linkedin_anon.jsonl
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.181.207, 173.194.193.207, 173.194.194.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.181.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 179851696 (172M) [application/octet-stream]
Saving to: ‘linkedin_anon.jsonl’

linkedin_anon.jsonl 100%[===================>] 171.52M   168MB/s    in 1.0s    

2025-09-29 18:55:42 (168 MB/s) - ‘linkedin_anon.jsonl’ saved [179851696/179851696]



## Access Patterns

Let's create two data structures, an integer list and a dictionary (hash map).  Each will have the same data.

In [2]:
intlist = []
for i in range(0,5000000):
  intlist.append((i+1,'a value'))

intdict = {}
for i in range(0,5000000):
  intdict[i] = ((i+1,'a value'))

In [3]:
%%time
count = 0
for i in range(0,len(intlist)):
  count += intlist[i][0]

CPU times: user 1.51 s, sys: 3.58 ms, total: 1.52 s
Wall time: 1.56 s


In [4]:
%%time
count = 0
for i in range(0,len(intdict)):
  count += intdict[i][0]

CPU times: user 2 s, sys: 5.58 ms, total: 2 s
Wall time: 2.09 s


In [5]:
%%time
# All 50,000+ records from linkedin
linked_in = open('linkedin_anon.jsonl')

copied_data = open('linkedin_anon_copy.jsonl','w', buffering=1)

count = 0
for repeat in range(0,10):
  linked_in.seek(0)
  for line in linked_in:
    count += 1
    copied_data.write(line)

print (f"Copied {count} records")

Copied 500000 records
CPU times: user 2.92 s, sys: 2.91 s, total: 5.83 s
Wall time: 10.3 s


In [6]:
%%time
# All 50,000+ records from linkedin
linked_in = open('linkedin_anon.jsonl')

copied_data = open('linkedin_anon_copy.jsonl','w', buffering=4096)

count = 0
for repeat in range(0,10):
  linked_in.seek(0)
  for line in linked_in:
    count += 1
    copied_data.write(line)

print (f"Copied {count} records")

Copied 500000 records
CPU times: user 2.41 s, sys: 2.89 s, total: 5.3 s
Wall time: 14.3 s


# Big Data Takes a Long Time to Process

Now that we've seen how to do fairly complex queries over data in relations, we'll "pop back" to our big data example, which is the LinkedIn dataset.  Recall that we had a segment of the LinkedIn input file in our previous examples earlier in this module.

In [7]:
!pip3 install lxml
!pip3 install duckdb

In [8]:
import pandas as pd
import numpy as np

# JSON parsing
import json

# HTML parsing
from lxml import etree
import urllib

# DuckDB RDBMS
import duckdb

# Time conversions
import time

In [9]:
%%time
# 50,000 records from linkedin
linked_in = open('linkedin_anon.jsonl')

people = []

for line in linked_in:
    person = json.loads(line)
    people.append(person)

people_df = pd.DataFrame(people)
people_df[people_df['industry'] == 'Medical Devices']

CPU times: user 4.49 s, sys: 547 ms, total: 5.04 s
Wall time: 8.95 s


,_id,name,locality,skills,industry,summary,url,education,group,interval,experience,specilities,events,interests,honors
0,moist-vodka,"{'family_name': 'Post', 'given_name': 'Belvede...",United States,"[Key Account Development, Strategic Planning, ...",Medical Devices,SALES MANAGEMENT / BUSINESS DEVELOPMENT / PROJ...,https://www.linkedin.com/in/moist-vodka,None,None,NaN,None,None,None,None,None
161,icy-bug,"{'family_name': 'Boyle', 'given_name': 'Simoni...",China,"[ISO 13485, Medical Devices]",Medical Devices,None,https://www.linkedin.com/in/icy-bug,"[{'start': '1998', 'end': '2001', 'name': 'Ton...","{'affilition': ['Beckman Coulter', 'Biomedical...",0.0,[{'org': 'Beckman Coulter Laboratory Systems S...,None,"[{'from': 'Beckman Coulter', 'to': 'Beckman Co...",None,None
335,pleasant-capital,"{'family_name': 'Fleming', 'given_name': 'Alfr...",Greater Nashville Area,None,Medical Devices,"Whether achieving new highs in medical sales, ...",https://www.linkedin.com/in/pleasant-capital,"[{'start': '1992', 'major': 'Criminal Justice,...",{'member': 'National Football League Players A...,32.0,"[{'org': 'Intuitive Surgical', 'title': 'Clini...","Customer Service, Sales Growth, Direct Sales, ...","[{'from': 'National Football League', 'to': 'C...",None,"[MERCK & CO., INC:, • Award of Excellence-2009..."
364,sour-database,"{'family_name': 'Donald', 'given_name': 'Belve...","Rochester, New York Area","[Process Validation, Quality Systems, ISO 1348...",Medical Devices,Change agent and proactive leader that drives ...,https://www.linkedin.com/in/sour-database,"[{'start': '2012', 'end': '2012', 'name': 'Smi...","{'member': 'ISPE, ASQ, PDA', 'affilition': ['A...",45.0,"[{'org': 'Ortho Clinical Diagnostics', 'title'...",None,"[{'from': 'Steris', 'to': 'Calgon Vestal', 'ti...",None,"[CQA, CQM, Sr. Member, ASQ]"
467,rectilinear-canyon,"{'family_name': 'Lyon', 'given_name': 'Alfred'}","Orange County, California Area","[Licensing, New Business Development, Relation...",Medical Devices,Contact –email: johnstroh@verizon.netmobile: 7...,https://www.linkedin.com/in/rectilinear-canyon,"[{'start': '1978', 'major': 'Graduate Business...",{'member': 'Council Member OCTANe Biomedical L...,25.0,"[{'org': 'Nanospectra Biosciences', 'desc': 'A...",None,"[{'from': 'AREAS OF INTEREST & EXPERTISE', 'to...","John Stroh – President, CEO, COO, CFO, Directo...",[Judge of the Business Plan Competition at the...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49045,congruent-enamel,"{'family_name': 'Murdoch', 'given_name': 'Cadb...",Greater Atlanta Area,"[Managed Care, Sales Effectiveness]",Medical Devices,"15 years of proven success in sales, sales tra...",https://www.linkedin.com/in/congruent-enamel,"[{'start': '1995', 'major': 'Biology, Chemistr...","{'affilition': ['Abbott Alumni', 'Abbott Labor...",36.0,[{'org': 'Anesthesia Healthcare Partners® (AHP...,Situational Leadership. Strong Customer-Needs-...,"[{'from': 'Abbott Laboratories', 'to': 'Sanofi...",None,None
49180,kind-locus,"{'family_name': 'Douglas', 'given_name': 'Bert...",Dallas/Fort Worth Area,"[Personnel Management, Accounting, Employee Re...",Medical Devices,None,https://www.linkedin.com/in/kind-locus,"[{'major': 'Healthcare Management', 'end': '20...",None,26.0,"[{'org': 'St. Jude Medical', 'title': 'Sr. Man...",None,"[{'from': 'Gables Residential', 'to': 'Matrix ...",None,None
49450,lower-prior,"{'family_name': 'Carmichael', 'given_name': 'J...",Greater New York City Area,None,Medical Devices,"Currently, Director of Human Resources at Life...",https://www.linkedin.com/in/lower-prior,"[{'start': '2009', 'end': '2009', 'name': 'Pen...","{'member': 'SHRM, World at Work', 'affilition'...",24.0,"[{'org': 'LifeCell Corporation', 'title': 'Dir...",None,"[{'from': 'Johnson & Johnson Corporate', 'to':...","golf, skiing, traveling with my family","[J&J Standard of Leadership Award 2007, 2004, ..."
49683,deterministic-ligature,"{'family_name': 'Murray', 'given_name': 'Figaro'}",Greater Chicago Are

In [10]:
%%time
# 500,000 records from linkedin
linked_in = open('linkedin_anon.jsonl')

people = []

for line in linked_in:
    person = json.loads(line)
    if 'industry' in person and person['industry'] == 'Medical Devices':
        people.append(person)

people_df = pd.DataFrame(people)
people_df

CPU times: user 1.92 s, sys: 45.5 ms, total: 1.96 s
Wall time: 1.97 s


,_id,name,locality,skills,industry,summary,url,education,group,interval,experience,specilities,events,interests,honors
0,moist-vodka,"{'family_name': 'Post', 'given_name': 'Belvede...",United States,"[Key Account Development, Strategic Planning, ...",Medical Devices,SALES MANAGEMENT / BUSINESS DEVELOPMENT / PROJ...,https://www.linkedin.com/in/moist-vodka,None,None,NaN,None,None,None,None,None
1,icy-bug,"{'family_name': 'Boyle', 'given_name': 'Simoni...",China,"[ISO 13485, Medical Devices]",Medical Devices,None,https://www.linkedin.com/in/icy-bug,"[{'start': '1998', 'end': '2001', 'name': 'Ton...","{'affilition': ['Beckman Coulter', 'Biomedical...",0.0,[{'org': 'Beckman Coulter Laboratory Systems S...,None,"[{'from': 'Beckman Coulter', 'to': 'Beckman Co...",None,None
2,pleasant-capital,"{'family_name': 'Fleming', 'given_name': 'Alfr...",Greater Nashville Area,None,Medical Devices,"Whether achieving new highs in medical sales, ...",https://www.linkedin.com/in/pleasant-capital,"[{'start': '1992', 'major': 'Criminal Justice,...",{'member': 'National Football League Players A...,32.0,"[{'org': 'Intuitive Surgical', 'title': 'Clini...","Customer Service, Sales Growth, Direct Sales, ...","[{'from': 'National Football League', 'to': 'C...",None,"[MERCK & CO., INC:, • Award of Excellence-2009..."
3,sour-database,"{'family_name': 'Donald', 'given_name': 'Belve...","Rochester, New York Area","[Process Validation, Quality Systems, ISO 1348...",Medical Devices,Change agent and proactive leader that drives ...,https://www.linkedin.com/in/sour-database,"[{'start': '2012', 'end': '2012', 'name': 'Smi...","{'member': 'ISPE, ASQ, PDA', 'affilition': ['A...",45.0,"[{'org': 'Ortho Clinical Diagnostics', 'title'...",None,"[{'from': 'Steris', 'to': 'Calgon Vestal', 'ti...",None,"[CQA, CQM, Sr. Member, ASQ]"
4,rectilinear-canyon,"{'family_name': 'Lyon', 'given_name': 'Alfred'}","Orange County, California Area","[Licensing, New Business Development, Relation...",Medical Devices,Contact –email: johnstroh@verizon.netmobile: 7...,https://www.linkedin.com/in/rectilinear-canyon,"[{'start': '1978', 'major': 'Graduate Business...",{'member': 'Council Member OCTANe Biomedical L...,25.0,"[{'org': 'Nanospectra Biosciences', 'desc': 'A...",None,"[{'from': 'AREAS OF INTEREST & EXPERTISE', 'to...","John Stroh – President, CEO, COO, CFO, Directo...",[Judge of the Business Plan Competition at the...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,congruent-enamel,"{'family_name': 'Murdoch', 'given_name': 'Cadb...",Greater Atlanta Area,"[Managed Care, Sales Effectiveness]",Medical Devices,"15 years of proven success in sales, sales tra...",https://www.linkedin.com/in/congruent-enamel,"[{'start': '1995', 'major': 'Biology, Chemistr...","{'affilition': ['Abbott Alumni', 'Abbott Labor...",36.0,[{'org': 'Anesthesia Healthcare Partners® (AHP...,Situational Leadership. Strong Customer-Needs-...,"[{'from': 'Abbott Laboratories', 'to': 'Sanofi...",None,None
233,kind-locus,"{'family_name': 'Douglas', 'given_name': 'Bert...",Dallas/Fort Worth Area,"[Personnel Management, Accounting, Employee Re...",Medical Devices,None,https://www.linkedin.com/in/kind-locus,"[{'major': 'Healthcare Management', 'end': '20...",None,26.0,"[{'org': 'St. Jude Medical', 'title': 'Sr. Man...",None,"[{'from': 'Gables Residential', 'to': 'Matrix ...",None,None
234,lower-prior,"{'family_name': 'Carmichael', 'given_name': 'J...",Greater New York City Area,None,Medical Devices,"Currently, Director of Human Resources at Life...",https://www.linkedin.com/in/lower-prior,"[{'start': '2009', 'end': '2009', 'name': 'Pen...","{'member': 'SHRM, World at Work', 'affilition'...",24.0,"[{'org': 'LifeCell Corporation', 'title': 'Dir...",None,"[{'from': 'Johnson & Johnson Corporate', 'to':...","golf, skiing, traveling with my family","[J&J Standard of Leadership Award 2007, 2004, ..."
235,deterministic-ligature,"{'family_name': 'Murray', 'given_name': 'Figaro'}",Greater Chicago Area,"[Process Deve

## SQL query without an index

SQL databases will automatically "push down" selection and projection where feasible.  They also don't need to parse.

Let's load people_df into tables as per our prior notebook.

In [11]:
'''
Simple code to pull out data from JSON and load into DuckDB.
'''
import ast

linked_in = open('linkedin_anon.jsonl')

START = 0
LIMIT = 50000

def get_df(rel):
    ret = pd.DataFrame(rel)
    return ret

lines = []
i = 1
for line in linked_in:
    if i > START + LIMIT:
        break
    elif i >= START:
        person = json.loads(line)

        lines.append(person)
    i = i + 1

people_df = get_df(pd.DataFrame(lines))



In [12]:
people_df

,_id,name,locality,skills,industry,summary,url,education,group,interval,experience,specilities,events,interests,honors
0,moist-vodka,"{'family_name': 'Post', 'given_name': 'Belvede...",United States,"[Key Account Development, Strategic Planning, ...",Medical Devices,SALES MANAGEMENT / BUSINESS DEVELOPMENT / PROJ...,https://www.linkedin.com/in/moist-vodka,None,None,NaN,None,None,None,None,None
1,adagio-catalyst,"{'family_name': 'Watt', 'given_name': 'Brunton'}","Antwerp Area, Belgium","[Molecular Biology, Biomarkers]",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,"[{'start': '2008', 'major': 'Economics', 'end'...","{'affilition': ['ASMALLWORLD.net', 'Biomarker ...",20.0,"[{'org': 'Johnson and Johnson', 'title': 'Seni...","Biomarkers in Oncology, Cancer Genomics, Molec...","[{'from': 'Sahlgrenska University Hospital', '...",None,None
2,tart-acorn,"{'family_name': 'Hannay', 'given_name': 'Passe...","San Francisco, California","[DNA, Nanotechnology, Molecular Biology, Softw...",Research,I am interested in inventing new methods to co...,https://www.linkedin.com/in/tart-acorn,"[{'major': 'Biophysics', 'end': '2009', 'name'...",None,0.0,"[{'org': 'UCSF', 'title': 'Assistant Professor...",None,[{'from': 'Wyss Institute for Biologically Ins...,"personal genomics, nanotechnology",None
3,objective-riesling,"{'family_name': 'Carnegie', 'given_name': 'Pas...",San Francisco Bay Area,None,Information Technology and Services,OBJECTIVE<Primary> Work on an interesting and ...,https://www.linkedin.com/in/objective-riesling,None,"{'affilition': ['Big Data, Low Latency', 'Expe...",5.0,"[{'org': '<Online Recruiting Company>', 'desc'...",None,"[{'from': '<Employee Benefits, Administration ...",None,None
4,generative-amberjack,"{'family_name': 'Duncan', 'given_name': 'Merri...","Chennai Area, India","[Program Management, French, Avionics, Embedde...",Aviation & Aerospace,"Experience in Avionics Systems, Embedded Syste...",https://www.linkedin.com/in/generative-amberjack,"[{'start': '1988', 'end': '1989', 'name': 'Eco...",{'member': 'Member of Project Management Insti...,NaN,None,None,None,"Literature, Philosophy, Music",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,glowing-flush,"{'family_name': 'Kincaid', 'given_name': 'Pass...",Greater Chicago Area,"[Spanish-speaking, Cicerone-Certified Beer Ser...",Marketing and Advertising,Sales and marketing professional specializing ...,https://www.linkedin.com/in/glowing-flush,"[{'start': '2007', 'major': 'PR and Advertisin...",None,5.0,"[{'org': 'Louis Glunz Beer Inc.', 'title': 'On...",None,"[{'from': 'Peet's Coffee and Tea', 'to': 'Expl...","craft beer industry, coffee industry, running,...",None
49996,grouchy-flight,"{'family_name': 'Ogilvy', 'given_name': 'Bulli...",Greater Atlanta Area,None,Financial Services,Accomplished business development manager expe...,https://www.linkedin.com/in/grouchy-flight,"[{'major': 'Marketing Focus', 'end': '2008', '...","{'member': 'Sigma Chi Fraternity', 'affilition...",14.0,"[{'org': 'Georgia-Pacific LLC', 'title': 'Acco...",None,"[{'from': 'Bayer Advanced', 'to': 'BBDO', 'tit...",None,[National Deans List]
49997,dense-bell,"{'family_name': 'Macdougall', 'given_name': 'B...","Calgary, Canada Area","[Project Management, Electrical Engineering, M...",Design,Brad Gibson is a recognized expert in power qu...,https://www.linkedin.com/in/dense-bell,[{'major': 'Engineering Physics (Solid State E...,"{'member': 'IEEE, APEGGA, APEGBC, PEO, APEGS, ...",42.0,"[{'org': 'DIALOG', 'desc': 'Electrical Enginee...","Data center design, high reliability power, po...","[{'from': 'Current Thinking Inc.', 'to': 'The ...",None,None
49998,brave-hoops,"{'family_name': 'Forsyth', 'given_name': 'Cadb...",San Francisco Bay Area,"[Corporate Social Responsibility, Public Polic...",Public Policy,Brad Kane's multi-faceted career in the govern...,https://www.linkedin.com/in/brave-hoops,"[{'major': 'Law', 'end':

In [13]:
def get_nested_dict(rel, name):
  # This evaluates the string that describes the dictionary, as a dictionary
  # definition
  ret = rel.copy()
  # ret[name] = rel[name].map(lambda x: ast.literal_eval(x) if len(x) else np.NaN)
  ret = ret.dropna()
  # This joins rows on the index
  return ret.drop(columns=name).join(pd.DataFrame(ret[name].tolist()))

def get_nested_list(rel, name):
  ret = rel.copy()
  ret = ret.dropna().explode(name).dropna()
  ret = ret.join(pd.DataFrame(ret[name].tolist())).drop(columns=name).drop_duplicates()
  return ret.rename(columns={0: name})

def get_nested_list_dict(rel, name):
  ret = rel.copy()

  ret = ret.dropna().explode(name)

  exploded_pairs = pd.DataFrame(ret.apply(lambda x: {'_id': x['_id']} | x[name] if isinstance(x[name], dict) else {'_id': x['_id']}, axis=1).tolist())

  return ret.merge(exploded_pairs, on='_id').drop(columns=name)
  #pd.DataFrame(ret[name].tolist())).drop(columns=name).drop_duplicates()

# Take the lists, drop any blank strings
specialties_df = people_df[['_id','specilities']].explode('specilities').rename(columns={'_id': 'person'})
specialties_df.dropna(inplace=True)
interests_df = people_df[['_id','interests']].explode('interests').rename(columns={'_id': 'person'})
interests_df.dropna(inplace=True)

names_df = get_nested_dict(people_df[['_id','name']], 'name')

education_df = get_nested_list_dict(people_df[['_id','education']], 'education')
experience_df = get_nested_list_dict(people_df[['_id','experience']], 'experience')
skills_df = get_nested_list(people_df[['_id','skills']], 'skills')
honors_df = get_nested_list(people_df[['_id','honors']], 'honors')
events_df = get_nested_list_dict(people_df[['_id','events']], 'events')

groups_df = get_nested_dict(people_df[['_id','group']], 'group')

people_only_df = people_df.drop(columns=['name','education','group','skills','experience','honors','events','specilities','interests'])

In [14]:
## This is just to reset things so we don't have an index
conn = duckdb.connect('linkedin.db')
conn.execute('BEGIN TRANSACTION')
conn.execute('DROP TABLE IF EXISTS people')
conn.execute('DROP INDEX IF EXISTS people_industry')
conn.execute('CREATE TABLE people AS SELECT * FROM people_df')
conn.execute('CREATE TABLE education AS SELECT * FROM education_df')
conn.execute('CREATE TABLE experience AS SELECT * FROM experience_df')
conn.execute('CREATE TABLE skills AS SELECT * FROM skills_df')
conn.execute('CREATE TABLE honors AS SELECT * FROM honors_df')
conn.execute('CREATE TABLE events AS SELECT * FROM events_df')
conn.execute('CREATE TABLE groups AS SELECT * FROM groups_df')
conn.execute('CREATE TABLE specialties AS SELECT * FROM specialties_df')
conn.execute('CREATE TABLE interests AS SELECT * FROM interests_df')
conn.execute('COMMIT')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [15]:
%%time

conn.sql("""
  SELECT *
  FROM people JOIN experience ON people._id = experience._id
  WHERE industry='Medical Devices'""")

CPU times: user 401 µs, sys: 766 µs, total: 1.17 ms
Wall time: 2.01 ms


┌───────────────┬──────────────────────┬──────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

## Let's build an index now...

Our data is very small, so the index probably won't speed anything up at this scale. But it can be created and the database will use it *transparently*!


In [16]:
conn.execute('BEGIN TRANSACTION')
conn.execute('DROP INDEX IF EXISTS people_industry')
conn.execute("CREATE INDEX people_industry ON people(industry)")
conn.execute('COMMIT')

In [17]:
%%time
# Treat the view as a table, see what's there
conn.sql("""
 CREATE VIEW people_medicine AS
  SELECT *
  FROM people JOIN experience ON people._id = experience._id
  WHERE industry='Medical Devices'""")

conn.sql("""
  SELECT *
  FROM people_medicine""")

# In our tests, this was 5x faster!

CPU times: user 598 µs, sys: 2.94 ms, total: 3.54 ms
Wall time: 7.01 ms


┌───────────────┬──────────────────────┬──────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [18]:
%%time

conn.sql("""
  SELECT name.given_name, name.family_name
  FROM people
  WHERE name.given_name='Jeeves'""")

CPU times: user 1.07 ms, sys: 139 µs, total: 1.21 ms
Wall time: 2.8 ms


┌────────────┬─────────────┐
│ given_name │ family_name │
│  varchar   │   varchar   │
├────────────┼─────────────┤
│ Jeeves     │ Boyle       │
│ Jeeves     │ Maclean     │
│ Jeeves     │ Riddell     │
│ Jeeves     │ Blair       │
│ Jeeves     │ Macduff     │
│ Jeeves     │ Riddell     │
│ Jeeves     │ Rollo       │
│ Jeeves     │ Davidson    │
│ Jeeves     │ Kennedy     │
│ Jeeves     │ Lough       │
│   ·        │   ·         │
│   ·        │   ·         │
│   ·        │   ·         │
│ Jeeves     │ Donald      │
│ Jeeves     │ Fleet       │
│ Jeeves     │ Jardine     │
│ Jeeves     │ Sinclair    │
│ Jeeves     │ Fergusson   │
│ Jeeves     │ Burnett     │
│ Jeeves     │ Buchanan    │
│ Jeeves     │ Fleet       │
│ Jeeves     │ Riddell     │
│ Jeeves     │ Lough       │
├────────────┴─────────────┤
│   3133 rows (20 shown)   │
└──────────────────────────┘

In [19]:
people_df2 = conn.sql('select * from people limit 500').df()
experience_df2 = conn.sql('select * from experience limit 5000').df()
skills_df2 = conn.sql('select * from skills limit 8000').df()

print ("%d people"%len(people_df2))
print ("%d experiences"%len(experience_df2))
print ("%d skills"%len(skills_df2))

500 people
5000 experiences
8000 skills


In [20]:
def merge(S,T,l_on,r_on):
    ret = []
    count = 0
    s_pos = S.columns.get_loc(l_on)
    t_pos = T.columns.get_loc(r_on)
    for s_index in range(0, len(S)):
        for t_index in range(0, len(T)):
            count = count + 1
            if S.iat[s_index, s_pos] == T.iat[t_index, t_pos]:
              ret.append(S.iloc[s_index].to_dict() | T.iloc[t_index].to_dict())

    print('Merge compared %d tuples'%count)
    return pd.DataFrame(ret)

In [21]:
%%time
# Here's a test join, with people and their experiences.  We can see how many
# comparisons are made

merge(people_df2, experience_df2, '_id', '_id')

Merge compared 2500000 tuples
CPU times: user 1min 33s, sys: 92.5 ms, total: 1min 33s
Wall time: 1min 35s


,_id,name,locality,skills,industry,summary,url,education,group,interval,experience,specilities,events,interests,honors,org,title,end,start,desc
0,adagio-catalyst,"{'family_name': 'Watt', 'given_name': 'Brunton'}","Antwerp Area, Belgium","[Molecular Biology, Biomarkers]",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,"[{'start': '2008', 'major': 'Economics', 'end'...","{'affilition': ['ASMALLWORLD.net', 'Biomarker ...",20.0,"[{'org': 'Johnson and Johnson', 'title': 'Seni...","Biomarkers in Oncology, Cancer Genomics, Molec...","[{'from': 'Sahlgrenska University Hospital', '...",None,None,Johnson and Johnson,"Senior Scientist, Oncology Biomarkers",Present,November 2009,Biomarker Leader for compounds in clinical dev...
1,adagio-catalyst,"{'family_name': 'Watt', 'given_name': 'Brunton'}","Antwerp Area, Belgium","[Molecular Biology, Biomarkers]",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,"[{'start': '2008', 'major': 'Economics', 'end'...","{'affilition': ['ASMALLWORLD.net', 'Biomarker ...",20.0,"[{'org': 'Johnson and Johnson', 'title': 'Seni...","Biomarkers in Oncology, Cancer Genomics, Molec...","[{'from': 'Sahlgrenska University Hospital', '...",None,None,Albert Einstein Medical Center,Associate at Dept of Molecular Genetics,None,September 2008,Single Cell Gene expression.
2,adagio-catalyst,"{'family_name': 'Watt', 'given_name': 'Brunton'}","Antwerp Area, Belgium","[Molecular Biology, Biomarkers]",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,"[{'start': '2008', 'major': 'Economics', 'end'...","{'affilition': ['ASMALLWORLD.net', 'Biomarker ...",20.0,"[{'org': 'Johnson and Johnson', 'title': 'Seni...","Biomarkers in Oncology, Cancer Genomics, Molec...","[{'from': 'Sahlgrenska University Hospital', '...",None,None,Columbia University,Associate Research Scientist,None,August 2006,Work on peptide to restore wt p53 function in ...
3,adagio-catalyst,"{'family_name': 'Watt', 'given_name': 'Brunton'}","Antwerp Area, Belgium","[Molecular Biology, Biomarkers]",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,"[{'start': '2008', 'major': 'Economics', 'end'...","{'affilition': ['ASMALLWORLD.net', 'Biomarker ...",20.0,"[{'org': 'Johnson and Johnson', 'title': 'Seni...","Biomarkers in Oncology, Cancer Genomics, Molec...","[{'from': 'Sahlgrenska University Hospital', '...",None,None,Memorial Sloan Kettering Cancer Center,Post Doctoral Research Fellow,None,January 2003,Molecular profiling of colorectal cancer.
4,adagio-catalyst,"{'family_name': 'Watt', 'given_name': 'Brunton'}","Antwerp Area, Belgium","[Molecular Biology, Biomarkers]",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,"[{'start': '2008', 'major': 'Economics', 'end'...","{'affilition': ['ASMALLWORLD.net', 'Biomarker ...",20.0,"[{'org': 'Johnson and Johnson', 'title': 'Seni...","Biomarkers in Oncology, Cancer Genomics, Molec...","[{'from': 'Sahlgrenska University Hospital', '...",None,None,Sahlgrenska University Hospital,Research Scientist,None,November 2001,Cancer Research at Dept of Surgery.Molecular p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,dry-dust,"{'family_name': 'Baird', 'given_name': 'Belved...","Shanghai City, China",None,Hospitality,None,https://www.linkedin.com/in/dry-dust,"[{'start': '1987', 'major': 'Hotel and caterin...","{'affilition': ['China Group 中国群', 'China Hote...",26.0,"[{'org': 'Morton's The Steakhouse', 'desc': 'G...",None,"[{'from': 'REIAIS DU CLAIN', 'to': 'La Champag...",None,"[Silver spoon 2007 ""business Lunch"" City week ...",PERSIA CAFÉ BAR OASIS (Opening),Assistant general Manager,None,February 2000,None
4996,dry-dust,"{'family_name': 'Baird', 'given_name': 'Belved...","Shanghai City, China"

In [22]:
# Let's find all people (by ID) who have Marketing as a skill

mbio_df = skills_df2[skills_df2['skills'] == 'Molecular Biology'].reset_index()[['_id']]
mbio_df

,_id
0,proper-strain
1,ancient-cluster
2,metallic-shark


In [23]:
%%time
merge(merge(people_df2, experience_df2, '_id', '_id'), mbio_df, '_id', '_id')

Merge compared 2500000 tuples
Merge compared 15000 tuples
CPU times: user 1min 33s, sys: 81 ms, total: 1min 33s
Wall time: 1min 36s


,_id,name,locality,skills,industry,summary,url,education,group,interval,experience,specilities,events,interests,honors,org,title,end,start,desc
0,proper-strain,"{'family_name': 'Strange', 'given_name': 'Jenk...","Paris Area, France","[JavaScript, HTML, CSS, WYSIWYG layout tools, ...",Internet,Conception de charte graphique pour des sites ...,https://www.linkedin.com/in/proper-strain,"[{'major': 'Public Relations, Advertising, and...",None,19.0,"[{'org': '01design', 'title': 'Owner', 'end': ...","XML, HTML, JavaScript, usability, CSS, WYSIWYG...","[{'from': 'Cyperus', 'to': 'Team Partners Grou...",None,None,01design,Owner,Present,March 2005,None
1,proper-strain,"{'family_name': 'Strange', 'given_name': 'Jenk...","Paris Area, France","[JavaScript, HTML, CSS, WYSIWYG layout tools, ...",Internet,Conception de charte graphique pour des sites ...,https://www.linkedin.com/in/proper-strain,"[{'major': 'Public Relations, Advertising, and...",None,19.0,"[{'org': '01design', 'title': 'Owner', 'end': ...","XML, HTML, JavaScript, usability, CSS, WYSIWYG...","[{'from': 'Cyperus', 'to': 'Team Partners Grou...",None,None,Team Partners Group,Webmaster,None,June 1998,None
2,proper-strain,"{'family_name': 'Strange', 'given_name': 'Jenk...","Paris Area, France","[JavaScript, HTML, CSS, WYSIWYG layout tools, ...",Internet,Conception de charte graphique pour des sites ...,https://www.linkedin.com/in/proper-strain,"[{'major': 'Public Relations, Advertising, and...",None,19.0,"[{'org': '01design', 'title': 'Owner', 'end': ...","XML, HTML, JavaScript, usability, CSS, WYSIWYG...","[{'from': 'Cyperus', 'to': 'Team Partners Grou...",None,None,Cyperus,Chargé des relations presse,None,March 1995,None
3,proper-strain,"{'family_name': 'Strange', 'given_name': 'Jenk...","Paris Area, France","[JavaScript, HTML, CSS, WYSIWYG layout tools, ...",Internet,Conception de charte graphique pour des sites ...,https://www.linkedin.com/in/proper-strain,"[{'major': 'Public Relations, Advertising, and...",None,19.0,"[{'org': '01design', 'title': 'Owner', 'end': ...","XML, HTML, JavaScript, usability, CSS, WYSIWYG...","[{'from': 'Cyperus', 'to': 'Team Partners Grou...",None,None,01design,Owner,Present,March 2005,None
4,proper-strain,"{'family_name': 'Strange', 'given_name': 'Jenk...","Paris Area, France","[JavaScript, HTML, CSS, WYSIWYG layout tools, ...",Internet,Conception de charte graphique pour des sites ...,https://www.linkedin.com/in/proper-strain,"[{'major': 'Public Relations, Advertising, and...",None,19.0,"[{'org': '01design', 'title': 'Owner', 'end': ...","XML, HTML, JavaScript, usability, CSS, WYSIWYG...","[{'from': 'Cyperus', 'to': 'Team Partners Grou...",None,None,Team Partners Group,Webmaster,None,June 1998,None
5,proper-strain,"{'family_name': 'Strange', 'given_name': 'Jenk...","Paris Area, France","[JavaScript, HTML, CSS, WYSIWYG layout tools, ...",Internet,Conception de charte graphique pour des sites ...,https://www.linkedin.com/in/proper-strain,"[{'major': 'Public Relations, Advertising, and...",None,19.0,"[{'org': '01design', 'title': 'Owner', 'end': ...","XML, HTML, JavaScript, usability, CSS, WYSIWYG...","[{'from': 'Cyperus', 'to': 'Team Partners Grou...",None,None,Cyperus,Chargé des relations presse,None,March 1995,None
6,proper-strain,"{'family_name': 'Strange', 'given_name': 'Jenk...","Paris Area, France","[JavaScript, HTML, CSS, WYSIWYG layout tools, ...",Internet,Conception de charte graphique pour des sites ...,https://www.linkedin.com/in/proper-strain,"[{'major': 'Public Relations, Advertising, and...",None,19.0,"[{'org': '01design', 'title': 'Owner', 'end': ...","XML, HTML, JavaScript, usability, CSS, WYSIWYG...","[{'from': 'Cyperus', 'to': 'Team Partners Grou...",None,None,01design,Owner,Present,March 2005,None
7,proper-strain,"{'family_name': 'Strange', 'given_name': 'Jenk...","Paris Area, France","[JavaScript, HTML, CSS, WYSIWYG layout tools, ...",Internet,Conception de charte graphique pour des sites ...,https://www.linkedin.com/in/proper-

In [24]:
%%time
merge(merge(people_df2, mbio_df, '_id', '_id'), experience_df2, '_id', '_id')

Merge compared 1500 tuples
Merge compared 5000 tuples
CPU times: user 211 ms, sys: 1.99 ms, total: 213 ms
Wall time: 213 ms


,_id,name,locality,skills,industry,summary,url,education,group,interval,experience,specilities,events,interests,honors,org,title,end,start,desc
0,proper-strain,"{'family_name': 'Strange', 'given_name': 'Jenk...","Paris Area, France","[JavaScript, HTML, CSS, WYSIWYG layout tools, ...",Internet,Conception de charte graphique pour des sites ...,https://www.linkedin.com/in/proper-strain,"[{'major': 'Public Relations, Advertising, and...",None,19.0,"[{'org': '01design', 'title': 'Owner', 'end': ...","XML, HTML, JavaScript, usability, CSS, WYSIWYG...","[{'from': 'Cyperus', 'to': 'Team Partners Grou...",None,None,01design,Owner,Present,March 2005,None
1,proper-strain,"{'family_name': 'Strange', 'given_name': 'Jenk...","Paris Area, France","[JavaScript, HTML, CSS, WYSIWYG layout tools, ...",Internet,Conception de charte graphique pour des sites ...,https://www.linkedin.com/in/proper-strain,"[{'major': 'Public Relations, Advertising, and...",None,19.0,"[{'org': '01design', 'title': 'Owner', 'end': ...","XML, HTML, JavaScript, usability, CSS, WYSIWYG...","[{'from': 'Cyperus', 'to': 'Team Partners Grou...",None,None,Team Partners Group,Webmaster,None,June 1998,None
2,proper-strain,"{'family_name': 'Strange', 'given_name': 'Jenk...","Paris Area, France","[JavaScript, HTML, CSS, WYSIWYG layout tools, ...",Internet,Conception de charte graphique pour des sites ...,https://www.linkedin.com/in/proper-strain,"[{'major': 'Public Relations, Advertising, and...",None,19.0,"[{'org': '01design', 'title': 'Owner', 'end': ...","XML, HTML, JavaScript, usability, CSS, WYSIWYG...","[{'from': 'Cyperus', 'to': 'Team Partners Grou...",None,None,Cyperus,Chargé des relations presse,None,March 1995,None
3,proper-strain,"{'family_name': 'Strange', 'given_name': 'Jenk...","Paris Area, France","[JavaScript, HTML, CSS, WYSIWYG layout tools, ...",Internet,Conception de charte graphique pour des sites ...,https://www.linkedin.com/in/proper-strain,"[{'major': 'Public Relations, Advertising, and...",None,19.0,"[{'org': '01design', 'title': 'Owner', 'end': ...","XML, HTML, JavaScript, usability, CSS, WYSIWYG...","[{'from': 'Cyperus', 'to': 'Team Partners Grou...",None,None,01design,Owner,Present,March 2005,None
4,proper-strain,"{'family_name': 'Strange', 'given_name': 'Jenk...","Paris Area, France","[JavaScript, HTML, CSS, WYSIWYG layout tools, ...",Internet,Conception de charte graphique pour des sites ...,https://www.linkedin.com/in/proper-strain,"[{'major': 'Public Relations, Advertising, and...",None,19.0,"[{'org': '01design', 'title': 'Owner', 'end': ...","XML, HTML, JavaScript, usability, CSS, WYSIWYG...","[{'from': 'Cyperus', 'to': 'Team Partners Grou...",None,None,Team Partners Group,Webmaster,None,June 1998,None
5,proper-strain,"{'family_name': 'Strange', 'given_name': 'Jenk...","Paris Area, France","[JavaScript, HTML, CSS, WYSIWYG layout tools, ...",Internet,Conception de charte graphique pour des sites ...,https://www.linkedin.com/in/proper-strain,"[{'major': 'Public Relations, Advertising, and...",None,19.0,"[{'org': '01design', 'title': 'Owner', 'end': ...","XML, HTML, JavaScript, usability, CSS, WYSIWYG...","[{'from': 'Cyperus', 'to': 'Team Partners Grou...",None,None,Cyperus,Chargé des relations presse,None,March 1995,None
6,proper-strain,"{'family_name': 'Strange', 'given_name': 'Jenk...","Paris Area, France","[JavaScript, HTML, CSS, WYSIWYG layout tools, ...",Internet,Conception de charte graphique pour des sites ...,https://www.linkedin.com/in/proper-strain,"[{'major': 'Public Relations, Advertising, and...",None,19.0,"[{'org': '01design', 'title': 'Owner', 'end': ...","XML, HTML, JavaScript, usability, CSS, WYSIWYG...","[{'from': 'Cyperus', 'to': 'Team Partners Grou...",None,None,01design,Owner,Present,March 2005,None
7,proper-strain,"{'family_name': 'Strange', 'given_name': 'Jenk...","Paris Area, France","[JavaScript, HTML, CSS, WYSIWYG layout tools, ...",Internet,Conception de charte graphique pour des sites ...,https://www.linkedin.com/in/proper-

In [25]:
%%time

conn.sql("""select distinct s._id,s.skills from people p join skills s on p._id=s._id join
                  experience ex on s._id=ex._id and s.skills='Molecular Biology'""")

CPU times: user 1.16 ms, sys: 0 ns, total: 1.16 ms
Wall time: 1.18 ms


┌─────────────────┬───────────────────┐
│       _id       │      skills       │
│     varchar     │      varchar      │
├─────────────────┼───────────────────┤
│ honest-margin   │ Molecular Biology │
│ ancient-cluster │ Molecular Biology │
│ organic-table   │ Molecular Biology │
│ matte-render    │ Molecular Biology │
│ honest-truss    │ Molecular Biology │
│ proper-strain   │ Molecular Biology │
│ metallic-shark  │ Molecular Biology │
└─────────────────┴───────────────────┘

In [26]:
%%time

conn.sql("""select distinct s._id,s.skills from skills s join
                  experience ex on s._id=ex._id join people p on p._id=s._id where s.skills='Molecular Biology'""")

CPU times: user 1.48 ms, sys: 0 ns, total: 1.48 ms
Wall time: 1.02 ms


┌─────────────────┬───────────────────┐
│       _id       │      skills       │
│     varchar     │      varchar      │
├─────────────────┼───────────────────┤
│ ancient-cluster │ Molecular Biology │
│ honest-margin   │ Molecular Biology │
│ proper-strain   │ Molecular Biology │
│ metallic-shark  │ Molecular Biology │
│ honest-truss    │ Molecular Biology │
│ organic-table   │ Molecular Biology │
│ matte-render    │ Molecular Biology │
└─────────────────┴───────────────────┘

In [27]:
conn.sql("select count(distinct _id) from skills where skills='Molecular Biology'")

┌─────────────────────┐
│ count(DISTINCT _id) │
│        int64        │
├─────────────────────┤
│                   8 │
└─────────────────────┘

In [28]:
# Join using a *hash map*
# from keys to (single) values
def merge_map(S,T,l_on,r_on):
    ret = []
    T_map = {}
    count = 0
    # Take each value in the r_on field, and
    # make a map entry for it
    t_pos = T.columns.get_loc(r_on)
    for t_index in range(0, len(T)):
        # Make sure we aren't overwriting an entry!
        if (T.iat[t_index,t_pos] not in T_map):
          T_map[T.iat[t_index,t_pos]] = [T.loc[t_index]]
        else:
          T_map[T.iat[t_index,t_pos]].append(T.loc[t_index])
        count = count + 1

    # Now find matches
    S2 = S.reset_index().drop(columns=['index'])
    for s_index in range(0, len(S2)):
        count = count + 1
        if S2.loc[s_index, l_on] in T_map:
          for item in T_map[S2.loc[s_index, l_on]]:
            ret.append(S2.loc[s_index].to_dict() | item.drop(labels=r_on).to_dict())

    print('Merge compared %d tuples'%count)
    return pd.DataFrame(ret)

In [29]:
%%time

# Here's a test join, with people and their experiences.  We can see how many
# comparisons are made
merge_map(experience_df2, people_df2, '_id', '_id')

Merge compared 5500 tuples
CPU times: user 1.68 s, sys: 8.06 ms, total: 1.69 s
Wall time: 1.69 s


,_id,org,title,end,start,desc,name,locality,skills,industry,summary,url,education,group,interval,experience,specilities,events,interests,honors
0,adagio-catalyst,Johnson and Johnson,"Senior Scientist, Oncology Biomarkers",Present,November 2009,Biomarker Leader for compounds in clinical dev...,"{'family_name': 'Watt', 'given_name': 'Brunton'}","Antwerp Area, Belgium","[Molecular Biology, Biomarkers]",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,"[{'start': '2008', 'major': 'Economics', 'end'...","{'affilition': ['ASMALLWORLD.net', 'Biomarker ...",20.0,"[{'org': 'Johnson and Johnson', 'title': 'Seni...","Biomarkers in Oncology, Cancer Genomics, Molec...","[{'from': 'Sahlgrenska University Hospital', '...",None,None
1,adagio-catalyst,Albert Einstein Medical Center,Associate at Dept of Molecular Genetics,None,September 2008,Single Cell Gene expression.,"{'family_name': 'Watt', 'given_name': 'Brunton'}","Antwerp Area, Belgium","[Molecular Biology, Biomarkers]",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,"[{'start': '2008', 'major': 'Economics', 'end'...","{'affilition': ['ASMALLWORLD.net', 'Biomarker ...",20.0,"[{'org': 'Johnson and Johnson', 'title': 'Seni...","Biomarkers in Oncology, Cancer Genomics, Molec...","[{'from': 'Sahlgrenska University Hospital', '...",None,None
2,adagio-catalyst,Columbia University,Associate Research Scientist,None,August 2006,Work on peptide to restore wt p53 function in ...,"{'family_name': 'Watt', 'given_name': 'Brunton'}","Antwerp Area, Belgium","[Molecular Biology, Biomarkers]",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,"[{'start': '2008', 'major': 'Economics', 'end'...","{'affilition': ['ASMALLWORLD.net', 'Biomarker ...",20.0,"[{'org': 'Johnson and Johnson', 'title': 'Seni...","Biomarkers in Oncology, Cancer Genomics, Molec...","[{'from': 'Sahlgrenska University Hospital', '...",None,None
3,adagio-catalyst,Memorial Sloan Kettering Cancer Center,Post Doctoral Research Fellow,None,January 2003,Molecular profiling of colorectal cancer.,"{'family_name': 'Watt', 'given_name': 'Brunton'}","Antwerp Area, Belgium","[Molecular Biology, Biomarkers]",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,"[{'start': '2008', 'major': 'Economics', 'end'...","{'affilition': ['ASMALLWORLD.net', 'Biomarker ...",20.0,"[{'org': 'Johnson and Johnson', 'title': 'Seni...","Biomarkers in Oncology, Cancer Genomics, Molec...","[{'from': 'Sahlgrenska University Hospital', '...",None,None
4,adagio-catalyst,Sahlgrenska University Hospital,Research Scientist,None,November 2001,Cancer Research at Dept of Surgery.Molecular p...,"{'family_name': 'Watt', 'given_name': 'Brunton'}","Antwerp Area, Belgium","[Molecular Biology, Biomarkers]",Pharmaceuticals,Ph.D. scientist with background in cancer rese...,https://www.linkedin.com/in/adagio-catalyst,"[{'start': '2008', 'major': 'Economics', 'end'...","{'affilition': ['ASMALLWORLD.net', 'Biomarker ...",20.0,"[{'org': 'Johnson and Johnson', 'title': 'Seni...","Biomarkers in Oncology, Cancer Genomics, Molec...","[{'from': 'Sahlgrenska University Hospital', '...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,dry-dust,PERSIA CAFÉ BAR OASIS (Opening),Assistant general Manager,None,February 2000,None,"{'family_name': 'Baird', 'given_name': 'Belved...","Shanghai City, China",None,Hospitality,None,https://www.linkedin.com/in/dry-dust,"[{'start': '1987', 'major': 'Hotel and caterin...","{'affilition': ['China Group 中国群', 'China Hote...",26.0,"[{'org': 'Morton's The Steakhouse', 'desc': 'G...",None,"[{'from': 'REIAIS DU CLAIN', 'to': 'La Champag...",None,"[Silver spoon 2007 ""business Lunch"" City week ..."
4996,dry-dust,BRASSERIE EXCHANGE ROYAL EXCHANGE THEATRE,Restaurant Manager,None,August 1

## Exercise

In [30]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Writing notebook-config.yaml


In [31]:
!pip3 install penngrader-client

In [32]:
#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
#TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 31763683 # YOUR PENN-ID GOES HERE AS AN INTEGER##PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO

In [33]:
%set_env HW_ID=cis5450_25f_HW9

env: HW_ID=cis5450_25f_HW9


In [34]:
import os
from penngrader.grader import *

grader = PennGrader('notebook-config.yaml', os.environ['HW_ID'], STUDENT_ID, STUDENT_ID)

PennGrader initialized with Student ID: 99999999

Make sure this correct or we will not be able to store your grade


Take the following query and use the `merge` or `merge_map` functions to execute it.  You can use Pandas to pre-apply or post-apply any filter conditions (selections) on dataframes.

```
SELECT _id, industry, skills
FROM people_df2 p JOIN skills_df2 s ON p._id = s._id
WHERE industry = 'Pharmaceuticals'
```

In [36]:
merged_df = people_df2.merge(skills_df2, on="_id")
print(merged_df.columns)

Index(['_id', 'name', 'locality', 'skills_x', 'industry', 'summary', 'url',
       'education', 'group', 'interval', 'experience', 'specilities', 'events',
       'interests', 'honors', 'skills_y'],
      dtype='object')


In [37]:
results_df = people_df2.merge(skills_df2, on="_id") \
                       .query("industry == 'Pharmaceuticals'")[["_id", "industry", "skills_y"]]

results_df


,_id,industry,skills_y
1,adagio-catalyst,Pharmaceuticals,Strategic Planning
25,all-architecture,Pharmaceuticals,Software Development
44,rude-turbine,Pharmaceuticals,Marketing Communications
71,faint-fencing,Pharmaceuticals,Public Relations
100,coral-submodule,Pharmaceuticals,Customer Relations
125,new-shiraz,Pharmaceuticals,MIS
171,crunchy-spitz,Pharmaceuticals,5S
196,fuchsia-quanta,Pharmaceuticals,Branded Content
255,roaring-zoom,Pharmaceuticals,Social Media Marketing
258,rectilinear-scuba,Pharmaceuticals,Microsoft Excel


In [38]:
grader.grade('pharma', results_df)

Correct! You earned 1/1 points. You are a star!

Your submission has been successfully recorded in the gradebook.
